# Particle Filtering Assignment
This assignment deals with particle filtering tutorials from the stone soup website

## Tutorial 1: Kalman Filter

In [1]:
import numpy as np
from datetime import datetime, timedelta

### Near constant velocity example
This example is for a simple "near constant velocity", which is that a target moving at constant velocity with some added noise.

We need these inputs:

In [2]:
from stonesoup.types.groundtruth import GroundTruthPath, GroundTruthState
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, ConstantVelocity

# Fix our random seed for a repeatable example
np.random.seed(1991)

In [3]:
# Start the clock
start_time = datetime.now()

# Create 2D transistion model
q_x = 0.05
q_y = 0.05
transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(q_x), ConstantVelocity(q_y)])

# Generate the truth path
truth_path = GroundTruthPath([GroundTruthState([0, 1, 0, 1], timestamp=start_time)])

num_steps = 20
for k in range(1, num_steps + 1):
    truth_path.append(GroundTruthState(
        transition_model.function(truth_path[k-1], noise=True, time_interval=timedelta(seconds=1)),
        timestamp=start_time+timedelta(seconds=k)))

After the ground truth path has been generated let us plot the results

In [4]:
from stonesoup.plotter import Plotterly
plotter = Plotterly()
plotter.plot_ground_truths(truth_path, [0, 2])
plotter.fig

In [5]:
print(transition_model.matrix(time_interval=timedelta(seconds=1)))
print(transition_model.covar(time_interval=timedelta(seconds=1)))

[[1. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 1.]
 [0. 0. 0. 1.]]
[[0.01666667 0.025      0.         0.        ]
 [0.025      0.05       0.         0.        ]
 [0.         0.         0.01666667 0.025     ]
 [0.         0.         0.025      0.05      ]]


### Simulating some measurements
We use a linear sensor model that can measure position (but not velocity) of a target such that:

$$
\mathbf{z}_k = H_k \mathbf{x}_k + \mathbf{v}_k \\
\mathbf{v}_k \sim \mathcal{N}(0, R)
$$

where the matrices

$$
H_k = \begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0
\end{bmatrix} \\

R = \begin{bmatrix}
1 & 0 \\
0 & 1
\end{bmatrix} \omega
$$

We set $\omega = 5$ initially

In [6]:
# We need the Detection type and a LinearGaussian measurement model
from stonesoup.types.detection import Detection
from stonesoup.models.measurement.linear import LinearGaussian

Let us set the linear Gaussian measurement by indicating the number of dimensions in the state vector and the dimensions that are measured (so $H_k$), and the noise covariance matrix, $R_k$.

In [7]:
w = 5
R = np.array([[1, 0],  # Covariance matrix for Gaussian PDF
              [0, 1]]) * w

measurement_model = LinearGaussian(
    ndim_state=4,  # Number of state dimensions (position and velocity in 2D)
    mapping=(0, 2),  # Mapping measurement vector index to state index
    noise_covar=R
    )

print(f"H_k =\n{measurement_model.matrix()}")
print(f"R   =\n{R}")

H_k =
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]]
R   =
[[5 0]
 [0 5]]


Now generate the measurements and results

In [8]:
# Generate the measurements
measurements = []
for state in truth_path:
    measurement = measurement_model.function(state, noise=True)
    measurements.append(Detection(measurement,
                                  timestamp=state.timestamp,
                                  measurement_model=measurement_model))

# Plot thie results
plotter.plot_measurements(measurements, [0, 2])
plotter.fig

### Construct a Kalman Filter

In [9]:
# Construct a Kalman predictor and updater
from stonesoup.predictor.kalman import KalmanPredictor
predictor = KalmanPredictor(transition_model)

from stonesoup.updater.kalman import KalmanUpdater
updater = KalmanUpdater(measurement_model)

# Run the Kalman filter
from stonesoup.types.state import GaussianState
prior = GaussianState([[0], [1], [0], [1]], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)

from stonesoup.types.hypothesis import SingleHypothesis

from stonesoup.types.track import Track
track = Track()
for measurement in measurements:
    prediction = predictor.predict(prior, timestamp=measurement.timestamp)
    hypothesis = SingleHypothesis(prediction, measurement)  # Group a prediction and measurement
    post = updater.update(hypothesis)
    track.append(post)
    prior = track[-1]

In [10]:
plotter.plot_tracks(track, [0, 2], uncertainty=True)
plotter.fig

### Observations
- The Kalman filter has 2 steps:
    - Prediction step: we try and evaluate the priori estimate and covariance matrices, where $\mathbf{x}_{k|k-1}$ and $P_{k|k-1}$ are the priori estimate and coveriance respectively

    $$
    \mathbf{x}_{k|k-1} = F_k \mathbf{x}_{k-1} + B_k \mathbf{u}_k \\
    P_{k|k-1} = F_k P_{k-1} F_k^T + Q_k
    $$

    - Update step: We use the priori estimates to calculate the posterior estimates by incorporating the measurement as follows:

    $$
    \mathbf{x}_k = \mathbf{x}_{k|k-1} + K_k (\mathbf{z}_k - H_k \mathbf{x}_{k|k-1} ) \\
    P_k = P_{k|k-1} - K_k H_k P_{k|k-1}
    $$

    - and the Kalman gain, $K_k$ is 

    $$
    K_k = P_{k|k-1} H_k^T S_k^{-1} \\
    S_k = H_k P_{k|k-1} H_k^T + R_k
    $$

- $F_k$ is the *state transition matrix*
    - It's important to keep in mind $F$ is _not_ estimated by the Kalman filter, because it _is the time step_.